In [1]:
import cobra
from cobra.io import load_json_model
from cobra import Model, Reaction, Metabolite

import pandas as pd

import escher
from escher import Builder

# 1. Glucose

### Predict fluxes
Synthetic medium from https://www.sciencedirect.com/science/article/pii/S0014579307007235
<br>"minimal synthetic medium containing 48 mM Na2HPO4, 22 mM KH2PO4, 9 mM NaCl, 19 mM NH4Cl, 2 mM MgSO4, 0.1 mM CaCl2, 0.1 g/l of thiamine and 2 g/l of glucose"

In [5]:
# Load the model
model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

In [4]:
import cameo
from cameo import phenotypic_phase_plane

In [8]:
wt_solution = model.optimize()
growth = wt_solution.fluxes["BIOMASS_Ec_iML1515_core_75p37M"]
acetate_production = wt_solution.fluxes["EX_ac_e"]

In [18]:
p = phenotypic_phase_plane(model, variables=['BIOMASS_Ec_iML1515_core_75p37M'], objective='EX_ac_e')
p.plot(growth, points=[(growth, acetate_production)])

NameError: name 'plot' is not defined

In [31]:
# Load the model
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.38 # 0.1 g/L > 0.1/265.355 x 1000 = 0.376854 mM

# trace elements
value = 0.01
medium['EX_fe3_e'] = value
medium['EX_mn2_e'] = value
medium['EX_fe2_e'] = value
medium['EX_zn2_e'] = value
medium['EX_ni2_e'] = value
medium['EX_cu2_e'] = value
medium['EX_cobalt2_e'] = value
medium['EX_sel_e'] = value
medium['EX_mobd_e'] = value
medium['EX_tungs_e'] = value
medium['EX_slnt_e'] = value
medium['EX_salchsx_e'] = value

EcN_model.medium = medium

In [32]:
# Run parsimonius FBA
sol_glc = cobra.flux_analysis.pfba(EcN_model)

# Export to dataframe
sol_glc_df = sol_glc.to_frame()

### Import flux data

In [33]:
# Import the flux file and set index
flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,2,3))
flux_df = flux_df.dropna()
flux_df.set_index('ID', inplace=True)
flux_df.head()

,Reaction,glc_flux_mean,glc_flux_sd
ID,,,
AKGDH,AKG → Suc + CO2,0.523333,0.008505
FBA,FBP ↔ GAP + GAP,0.715000,0.025632
CS,OAA + AcCoA → Cit,0.592333,0.008505
EDA,6PG → GAP + Pyr,0.166667,0.037166
ENO,PGA ↔ PEP,1.511000,0.025632


In [34]:
# Merge dataframes
glc_flux_df = pd.merge(flux_df, sol_glc_df, left_index = True, right_index=True)
glc_flux_df['fluxes_corrected'] = glc_flux_df['fluxes']/15 # normalise for 15 glucose
glc_flux_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True) # Do not remove when normalising the 13C data
glc_flux_df

# Alternatively compare the flux with the corrected 13C data
# glc_flux_df['glc_flux_mean_corrected'] = glc_flux_df['glc_flux_mean']*15 # normalise for 15 glucose
# glc_flux_df['glc_flux_sd_corrected'] = glc_flux_df['glc_flux_sd']*15 # normalise for 15 glucose
# glc_flux_df

# Add the subsystem for each reaction and sort
for rxn_id in glc_flux_df.index.values:
    glc_flux_df.loc[rxn_id, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn_id).subsystem
glc_flux_df.sort_values('Subsystem')

,Reaction,glc_flux_mean,glc_flux_sd,fluxes_corrected,Subsystem
ME2,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
ME1,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
PPC,PEP + CO2 ↔ OAA,0.260000,0.01253,0.196367,Anaplerotic Reactions
AKGDH,AKG → Suc + CO2,0.523333,0.008505,0.000000,Citric Acid Cycle
ICDHyr,Cit → AKG + CO2,0.592333,0.008505,0.071323,Citric Acid Cycle
MDH,Mal ↔ OAA,0.457667,0.01365,0.000000,Citric Acid Cycle
CS,OAA + AcCoA → Cit,0.592333,0.008505,0.071323,Citric Acid Cycle
FUM,Suc → Mal,0.523333,0.007572,0.069174,Citric Acid Cycle
ENO,PGA ↔ PEP,1.511000,0.025632,1.629559,Glycolysis/Gluconeogenesis
PYK,PEP ↔ Pyr,0.203000,0.020809,0.000000,Glycolysis/Gluconeogenesis


### Visualise fluxes

In [35]:
# build
# builder = Builder(
#     map_name='iJO1366.Central metabolism',
#     model_name='iJO1366')

builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

In [36]:
builder

Builder()

In [37]:
builder.reaction_data = sol_glc.fluxes

### Save the dataframe

In [38]:
# Save as .csv
glc_flux_df.to_csv('../tables/glc_flux_df.csv')

# 2. Gluconate

In [39]:
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 0
medium['EX_glcn_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.1 # 0.1 g/L

EcN_model.medium = medium

In [40]:
# Calculate fluxes
sol_glcn = cobra.flux_analysis.pfba(EcN_model) # parsimonious fba
# sol_glcn = EcN_model.optimize() # fba

# Export to dataframe
sol_glcn_df = sol_glcn.to_frame()

In [41]:
# Import the flux file and set index
flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,4,5))
flux_df = flux_df.dropna()
flux_df.set_index('ID', inplace=True)
flux_df.head()

,Reaction,glcn_flux_mean,glcn_flux_sd
ID,,,
AKGDH,AKG → Suc + CO2,0.671417,0.006851
FBA,FBP ↔ GAP + GAP,0.088321,0.00648
CS,OAA + AcCoA → Cit,0.711340,0.006851
EDA,6PG → GAP + Pyr,0.808279,0.011769
ENO,PGA ↔ PEP,0.971385,0.00648


In [42]:
# Merge the 13C data with the model's flux predictions
glcn_flux_df = pd.merge(flux_df, sol_glcn_df, left_index = True, right_index=True)
glcn_flux_df['fluxes_corrected'] = glcn_flux_df['fluxes']/15 # normalise for 15 gluconate
glcn_flux_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True)

# Add the subsystem for each reaction and sort
for rxn_id in glcn_flux_df.index.values:
    glcn_flux_df.loc[rxn_id, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn_id).subsystem   
glcn_flux_df.sort_values('Subsystem')

,Reaction,glcn_flux_mean,glcn_flux_sd,fluxes_corrected,Subsystem
ME2,Mal → Pyr + CO2,0.005609,0.009716,0.000000,Anaplerotic Reactions
ME1,Mal → Pyr + CO2,0.005609,0.009716,0.000000,Anaplerotic Reactions
PPC,PEP + CO2 ↔ OAA,0.111706,0.009716,0.187316,Anaplerotic Reactions
AKGDH,AKG → Suc + CO2,0.671417,0.006851,0.000000,Citric Acid Cycle
ICDHyr,Cit → AKG + CO2,0.711340,0.006851,0.068036,Citric Acid Cycle
MDH,Mal ↔ OAA,0.665776,0.016548,0.000000,Citric Acid Cycle
CS,OAA + AcCoA → Cit,0.711340,0.006851,0.068036,Citric Acid Cycle
FUM,Suc → Mal,0.671417,0.006851,0.065986,Citric Acid Cycle
ENO,PGA ↔ PEP,0.971385,0.00648,1.403828,Glycolysis/Gluconeogenesis
PYK,PEP ↔ Pyr,0.826861,0.011014,0.303610,Glycolysis/Gluconeogenesis


In [43]:
builder_glcn = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

builder_glcn

Builder()

In [44]:
builder_glcn.reaction_data = sol_glcn.fluxes

### Test: Gluconate with forced ED pathway

In [45]:
# Restrict the flow through the PPP to 1/5th of the substrate
EcN_model.reactions.GND.upper_bound = 3

In [46]:
# Calculate fluxes
sol_glcn_ED = cobra.flux_analysis.pfba(EcN_model)
# sol_glcn = EcN_model.optimize()

# Export to dataframe
sol_glcn_ED_df = sol_glcn_ED.to_frame()

In [50]:
# Merge tables
glcn_flux_ED_df = pd.merge(glcn_flux_df, sol_glcn_ED_df, left_index = True, right_index=True)
glcn_flux_ED_df['fluxes_ED_corrected'] = glcn_flux_ED_df['fluxes']/15 # normalise for 15 gluconate

# Drop unused columns
glcn_flux_ED_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True)

glcn_flux_ED_df

,Reaction,glcn_flux_mean,glcn_flux_sd,fluxes_corrected,Subsystem,fluxes_ED_corrected
AKGDH,AKG → Suc + CO2,0.671417,0.006851,0.000000,Citric Acid Cycle,0.229057
FBA,FBP ↔ GAP + GAP,0.088321,0.00648,0.000000,Glycolysis/Gluconeogenesis,0.000000
CS,OAA + AcCoA → Cit,0.711340,0.006851,0.068036,Citric Acid Cycle,0.291886
EDA,6PG → GAP + Pyr,0.808279,0.011769,0.008169,Pentose Phosphate Pathway,0.800000
ENO,PGA ↔ PEP,0.971385,0.00648,1.403828,Glycolysis/Gluconeogenesis,0.892398
FUM,Suc → Mal,0.671417,0.006851,0.065986,Citric Acid Cycle,0.289993
RPI,6PG → R5P + CO2,0.197801,0.019392,-0.377150,Pentose Phosphate Pathway,-0.109645
TKT2,F6P ↔ E4P + E2,0.045930,0.006464,0.294044,Pentose Phosphate Pathway,0.032899
TKT1,R5P + E2 ↔ S7P,0.059300,0.006464,0.318118,Pentose Phosphate Pathway,0.055130
ICDHyr,Cit → AKG + CO2,0.711340,0.006851,0.068036,Citric Acid Cycle,0.291886


In [51]:
builder_glcn_ED = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

builder_glcn_ED

Builder()

In [52]:
builder_glcn_ED.reaction_data = sol_glcn_ED.fluxes

In [53]:
# Save the dataframe
glcn_flux_ED_df.to_csv('../tables/glcn_flux_ED_df.csv')